In [37]:
import numpy as np

from datetime import datetime

from fatiando.vis import mpl
from fatiando import mesher, gridder, utils
from fatiando.gravmag import prism, sphere
from fatiando.constants import G, SI2MGAL
from fatiando.utils import gaussian2d
from fatiando.vis import mpl, myv
from fatiando.gravmag.eqlayer import EQLGravity
from fatiando.inversion.regularization import Damping, LCurve


In [38]:
def A(xi,yi,zi,xk,yk,zk):
    raio = 1.0
    volume = 1.0 
    size = len(xi)
    #for i in range(size):
    rz = zi-zk
    rx = xi-xk
    ry = yi-yk
    r  = np.sqrt(rx**2+ry**2+rz**2)
    r3=r**3
    gx = -volume*G*rx/r3
    gy = -volume*G*ry/r3
    gz = -volume*G*rz/r3
    Ax = gx*SI2MGAL
    Ay = gy*SI2MGAL
    Az = gz*SI2MGAL
    return Ax,Ay,Az

In [39]:
def A_Fillipe(xi,yi,zi,xk,yk,zk):
    '''Calcula a componente vertical
    da atracao gravitacional exercida
    por uma fonte equivalente
    
    input : (xi,yi,zi) são coordenadas 
    da medida e (xk,yk,zk) são as 
    coordenadas das fontes.
    
    output: matriz de sensibilidade G
        
    '''    
    #C=6.67384*10**(-11)
    #print 'SI2MGAL', SI2MGAL
    #print 'G', G
    raio = 1.0
    volume = 1.0 #(4./3.)*np.pi*(raio**3)
    N=len(xi)
    M=len(xk)
    Ax = np.zeros((N,M), dtype='float')
    Ay = np.zeros((N,M), dtype='float')
    Az = np.zeros((N,M), dtype='float')
    #for que anda nas colunas (nas fontes)
    for j in range(M):
        #for que anda nas linhas (nos dados)
        for i in range(N):
            rz = zi[i]-zk[j]
            rx = xi[i]-xk[j]
            ry = yi[i]-yk[j]
            r  = np.sqrt(rx**2+ry**2+rz**2)
            r3=r**3
            gx = -volume*G*rx/r3
            gy = -volume*G*ry/r3
            gz = -volume*G*rz/r3
            Ax[i][j] = gx*SI2MGAL
            Ay[i][j] = gy*SI2MGAL
            Az[i][j] = gz*SI2MGAL
    return Ax, Ay, Az

In [40]:
start_time_total = datetime.now()

N_East  = 100
N_North = 100
area   = (-5000, 5000, -5000, 5000)
shape_obs = (N_North,N_East)

z_dado = -150.0

xi, yi, zi = gridder.regular(area, shape_obs, z=z_dado)


# Fontes Equivalentes 

In [41]:
plano_fonte = 200.
zk = np.zeros_like(zi) + plano_fonte
print zk

[ 200.  200.  200. ...,  200.  200.  200.]


## By Valeria: Jeito mais rápido de fazer as Matrizes de Sensibilidade das componentes x, y e z da esfera 

In [42]:
Npts = len(xi)
M    = len(zk)
print Npts, M
AX = np.empty((Npts,M),dtype =float)
AY = np.empty((Npts,M),dtype =float)
AZ = np.empty((Npts,M),dtype =float)

start_time_matrizes_valeria = datetime.now()

for j  in range(M):
    AX[:,j], AY[:,j], AZ[:,j] =A(xi,yi,zi,xi[j],yi[j],zk[j])

end_time_matrizes_valeria = datetime.now()
end_time_matrizes_valeria = datetime.now()
print('Duration: {}'.format(end_time_matrizes_valeria - start_time_matrizes_valeria))


10000 10000
Duration: 0:00:16.113657


## Calculo da Componente z via Fatiando

In [43]:
layer = mesher.PointGrid(area, plano_fonte, shape_obs)
AZ_Fatiando = np.empty((Npts,M),dtype =float)
for i, c in enumerate(layer):
    AZ_Fatiando[:,i] = sphere.gz(xi, yi, zi, [c], dens=1.)
    

In [44]:
np.allclose(AZ_Fatiando, AZ)

True

## By Fillipe: Jeito clássico de fazer as Matrizes de Sensibilidade das componentes x, y e z da esfera  (dois ninhos de loops)

In [45]:
start_time_matrizes_fillipe = datetime.now()

Gx,Gy,Gz = A_Fillipe(xi,yi,zi,xi,yi,zk)

end_time_matrizes_fillipe = datetime.now()
print('Duration: {}'.format(end_time_matrizes_fillipe - start_time_matrizes_fillipe))   

np.allclose(AZ_Fatiando, Gz)

Duration: 0:13:09.343097


True

## Compara as matrizes 

In [46]:
np.allclose(AX, Gx)

True

In [47]:
np.allclose(AY, Gy)


True

In [48]:
np.allclose(AZ, Gz)

True